1.继承 MODULE 类来构造模型

In [ ]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self.hidden = nn.Linear(784,256)
        self.act = nn.ReLU()
        self.output = nn.Linear (256,10)
    def forward(self,x):
        a = self.act(self.hidden(x))
        return self.output(a)

X = torch.rand(2,784)
net = MLP()
print(net(X))

tensor([[-0.1318,  0.2245,  0.0180,  0.0861, -0.3759, -0.0185, -0.1809,  0.0109,
         -0.1101,  0.1006],
        [-0.0654,  0.0245, -0.0501, -0.0126, -0.2898, -0.0471, -0.2208, -0.1613,
         -0.2569,  0.2383]], grad_fn=<AddmmBackward0>)


2.Moduel的子类

In [ ]:
from collections import OrderedDict  # 有序字典，确保层添加顺序
#Seqential类
class MySequential(nn.Module):
    # __init__: 初始化层
    def __init__(self, *args):
        super(MySequential, self).__init__()  # 调用父类nn.Module初始化
        if len(args) == 1 and isinstance(args[0], OrderedDict):  # 输入是OrderedDict
            for key, module in args[0].items():
                self.add_module(key, module)  # 添加模块到self._modules (OrderedDict)
        else:  # 输入是层列表/元组
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)  # 键=索引字符串

    # forward: 前向传播
    def forward(self, input):
        # self._modules返回OrderedDict，按添加顺序遍历
        for module in self._modules.values():
            input = module(input)  # 逐层计算，更新input
        return input
net = MySequential(  # 创建MySequential实例
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10),
)
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.0301,  0.0731,  0.0561, -0.0113, -0.0155, -0.0814, -0.1044,  0.1248,
         -0.1069,  0.0906],
        [-0.0249,  0.2178, -0.0007, -0.0096,  0.1099, -0.1489, -0.0043,  0.1677,
         -0.2439,  0.2372]], grad_fn=<AddmmBackward0>)

In [4]:
#ModuleList类
net = nn.ModuleList([nn.Linear(784, 256), nn.ReLU()])  # 1. 初始化列表
net.append(nn.Linear(256, 10))  # 2. append添加层
print(net[-1])  # 3. 索引访问最后一个
print(net)  # 4. 打印结构

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [5]:
#ModuleDict 类
net = nn.ModuleDict({
 'linear': nn.Linear(784, 256),
 'act': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10) # 添加
print(net['linear']) # 访问
print(net.output)
print(net)

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


3.构造复杂的模型

In [6]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)  # 调用父类初始化
        self.rand_weight = torch.rand((20, 20), requires_grad=False)  # 常数参数（不可训练）
        self.linear = nn.Linear(20, 20)  # 全连接层（可训练参数）

    def forward(self, x):
        x = self.linear(x)  # 第一Linear
        # 使用常数参数，以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)
        # 复用全连接层。等价于两个全连接层共享参数
        x = self.linear(x)  # 复用linear
        # 控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2  # 规范范数
        if x.norm().item() < 0.8:
            x *= 10  # 缩放
        return x.sum()  # 输出标量总和
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
out = net(X)
print(out)  # tensor([12.34, 15.67], grad_fn=<SumBackward0>)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)
tensor(15.8674, grad_fn=<SumBackward0>)


In [7]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)  # 调用父类初始化
        self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU())  # 内部Sequential子模块

    def forward(self, x):
        return self.net(x)  # 委托给内部Sequential
net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())  # 复合Sequential
X = torch.rand(2, 40)  # 随机输入：2样本，40维
print(net)  # 打印结构
net(X)      # 前向计算

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(6.2996, grad_fn=<SumBackward0>)